In [1]:
import sys
import numpy as np
import import_ipynb
from pathlib import Path
from scipy.sparse.linalg import spsolve

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Coaxial Cable

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_problem.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

O raio do condutor interno é ``a`` e do condutor externo é ``b``. O primeiro dielétrico, $\varepsilon_{1}$, preenche o anel circular entre os raios ``a`` e ``c``, enquanto o segundo dielétrico, $\varepsilon_{2}$, preenche o anel entre os raios ``c`` e ``b``.  

Entre o condutor interno e o externo é aplicada uma diferença de potencial igual a $V$.  

Seja $V = 1$ (potencial zero em ``a`` e um em ``b``), $a = 2 \, mm$, $b = 8 \, mm$, $c = 5 \, mm$, $\varepsilon_{r1} = 2$ e $\varepsilon_{r2} = 4$. Para este problema, além da convergência nas normas $L_2$ e da energia, calcule a convergência para o valor da capacitância do dispositivo.

# 10-noded cubic Triangular Elements, $P_3$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 3)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'inner_conductor'},
                       {'type': 'Dirichlet', 'tag': 102, 'value': 1.0, 'name': 'outer_conductor'}]

MATERIAL = [{'type': 'relative_electric_permittivity', 'tag': 201, 'value': 2, 'name': 'medium_1'},
                        {'type': 'relative_electric_permittivity', 'tag': 202, 'value': 4, 'name': 'medium_2'}]

# Create mesh from file geometry
create_domain.coaxial_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-3, view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='coaxial')
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='coaxial')

Model coaxial_domain_Triangle3 (2D)
Info     : 8 geometric entities
Info     : 4 Physical Groups
Info     : 2373 nodes in total
Info     : 506 2-D elements in total
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pre_processing\pictures\coaxial_meshed_domain_Triangle3.svg


### Figure 1: Coaxial meshed domain 

<figure>
    <img src="pre_processing/pictures/coaxial_meshed_domain_Triangle3.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Coaxial meshed domain.</figcaption>
</figure>

In [3]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

#### `apply_physics`

In [4]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 100,
  'conn_list': [296, 314, 295, 360, 361, 362, 363, 364, 365, 366],
  'material': {'type': 'relative_electric_permittivity',
   'tag': 201,
   'value': 2,
   'name': 'medium_1',
   'a_value': 1.77083756256e-11},
  'source': {'type': 'free_source', 'value': 0}}]

### Imposition of Boundary Conditions

In [5]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")

The entire domain has 2373 nodes: 2181 free nodes and 192 Dirichlet nodes.


### Reduced Global matrix $A_{gr}$ with boundary conditions

In [6]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (2181, 2181)


## Processor Module
### Reduced Global matrix

In [7]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)

## Post-Processor

In [8]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='coaxial')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p12_coaxial_electrostatics\pos_processing\pictures\coaxial_domain_solution_Triangle3.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh

<figure>
    <img src="pos_processing/pictures/coaxial_domain_solution_Triangle3.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.